In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
# Make the union of two dictionary without duplicates
def union_dict(d1, d2):
  union = d1.copy()
  for i in d1:
    for j in d2:
      if i == j:
        union[i] = list(set(d1[i] + d2[j]))
        break
  return union

## Adjacency matrix

In [2]:
# import dataset
adj_county = pd.read_csv ('county_adjacency2010.csv')
county_dist = pd.read_csv ('sf12010countydistance50miles.csv')
air_mobility = pd.read_csv('air_mobility.csv')

In [ ]:
# Create dictionary with adjacency county
county_dict = {}
for index, row in adj_county.iterrows():
  if row['fipscounty'] not in county_dict:
    county_dict[row['fipscounty']] = [int(row['fipsneighbor'])]
  else:
    county_dict[row['fipscounty']].append(int(row['fipsneighbor']))

for i in county_dict:
  county_dict[i].remove(i)

# Create dictionary with county with a maximum distance of 50 miles
dist_county_dict = {}
for index, row in county_dist.iterrows():
  if row['county1'] not in dist_county_dict:
    dist_county_dict[row['county1']] = [int(row['county2'])]
  else:
    dist_county_dict[row['county1']].append(int(row['county2']))

air = {}
for index, row in air_mobility.iterrows():
  if row['dest_fips'] not in air:
    air[row['dest_fips']] = [int(row['origin_fips'])]
  else:
    air[row['dest_fips']].append(int(row['origin_fips']))

In [ ]:
# Make the union with the three dictionaries
adj_dict = union_dict(county_dict, dist_county_dict)
adj_dict = union_dict(adj_dict, air)

In [ ]:
# Create the adjacency matrix
adj = np.zeros((len(adj_dict), len(adj_dict)))
# Create a list containing all the fips of all county
list_keys = list(adj_dict.keys())

# Fill the adjacency matrix with 1 if the the two counties are adjacent or have a distance lower of 50 miles
for i in range(len(list_keys)):
  for j in range(len(list_keys)):
    if list_keys[j] in adj_dict[list_keys[i]]:
      adj[i][j] = 1

In [ ]:
# Convert adjacency matrix to a dataframe
adj_df = pd.DataFrame(adj, columns = list_keys)
# Insert a new column with the fips of counties
adj_df.insert(0, "--", list_keys, allow_duplicates=True)
# Export
adj_df.to_csv('adjacency_matrix.csv', index=False)

## Vaccination dataset

In [ ]:
# import dataset
vaccination = pd.read_csv ('COVID-19_Vaccinations_in_the_United_States_County.csv')
# remove useless column
vaccination.drop(['MMWR_week','Administered_Dose1_Recip','Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_5Plus',
       'Administered_Dose1_Recip_5PlusPop_Pct','Administered_Dose1_Recip_12Plus','Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus','Administered_Dose1_Recip_18PlusPop_Pct','Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'Series_Complete_Yes','Series_Complete_Pop_Pct', 'Series_Complete_5Plus',
       'Series_Complete_5PlusPop_Pct', 'Series_Complete_5to17','Series_Complete_5to17Pop_Pct', 'Series_Complete_12Plus',
       'Series_Complete_12PlusPop_Pct', 'Series_Complete_18Plus','Series_Complete_18PlusPop_Pct', 'Series_Complete_65Plus',
       'Series_Complete_65PlusPop_Pct', 'Booster_Doses','Booster_Doses_Vax_Pct', 'Booster_Doses_12Plus',
       'Booster_Doses_12Plus_Vax_Pct', 'Booster_Doses_18Plus','Booster_Doses_18Plus_Vax_Pct', 'Booster_Doses_50Plus',
       'Booster_Doses_50Plus_Vax_Pct', 'Booster_Doses_65Plus','Booster_Doses_65Plus_Vax_Pct', 'SVI_CTGY',
       'Series_Complete_Pop_Pct_SVI', 'Series_Complete_5PlusPop_Pct_SVI','Series_Complete_5to17Pop_Pct_SVI', 'Series_Complete_12PlusPop_Pct_SVI',
       'Series_Complete_18PlusPop_Pct_SVI','Series_Complete_65PlusPop_Pct_SVI', 'Metro_status','Series_Complete_Pop_Pct_UR_Equity',
       'Series_Complete_5PlusPop_Pct_UR_Equity','Series_Complete_5to17Pop_Pct_UR_Equity','Series_Complete_12PlusPop_Pct_UR_Equity',
       'Series_Complete_18PlusPop_Pct_UR_Equity','Series_Complete_65PlusPop_Pct_UR_Equity', 'Booster_Doses_Vax_Pct_SVI',
       'Booster_Doses_12PlusVax_Pct_SVI', 'Booster_Doses_18PlusVax_Pct_SVI','Booster_Doses_65PlusVax_Pct_SVI', 'Booster_Doses_Vax_Pct_UR_Equity',
       'Booster_Doses_12PlusVax_Pct_UR_Equity','Booster_Doses_18PlusVax_Pct_UR_Equity','Booster_Doses_65PlusVax_Pct_UR_Equity','Census2019_5PlusPop', 
       'Census2019_5to17Pop', 'Census2019_12PlusPop','Census2019_18PlusPop', 'Census2019_65PlusPop'], inplace=True, axis=1)
# Export dataframe
vaccination.to_csv('vaccination_per_county.csv', index=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Population and Percentage of Vaccination

In [ ]:
census = pd.read_csv ('vaccination_per_county.csv')

last_index = census.loc[census['Date'] == '04/12/2022'].index
index_to_remove = np.arange(0, last_index[-1] + 1, 1)
census.drop(index_to_remove, axis=0, inplace=True)

for i in range(2020, 2023):
  for j in range(1, 13):
    if i == 2022 and j >= 5:
      break
    for k in range(1, 32):
      day = str(k)
      month = str(j)
      year = str(i)
      if len(day) == 1:
        day = '0'+ day
      if len(month) == 1:
        month = '0' + month
      old_date = month + '/' + day + '/' + year
      new_date = year + '-' + month + '-' + day
      index_to_change = census.loc[census['Date'] == old_date].index
      if len(index_to_change) > 0:
        census.loc[index_to_change, 'Date'] = new_date

census.to_csv('vaccination_per_county.csv', index=False)

In [ ]:
census2022 = census.iloc[:331580]
census2021 = census.iloc[331580:1529581]
census2020 = census.iloc[1529581:]

census2022.to_csv('vaccination_per_county_2022.csv', index=False)
census2021.to_csv('vaccination_per_county_2021.csv', index=False)
census2020.to_csv('vaccination_per_county_2020.csv', index=False)

In [6]:
population = {}
vax_22 = {}
vax_21 = {}
vax_20 = {}

for index, row in census2022.iterrows():
  if row['FIPS'] == 'UNK':
    continue
  if int(row['FIPS']) not in population:
    population[int(row['FIPS'])] = row['Census2019']

  if int(row['FIPS']) not in vax_22:
    vax_22[int(row['FIPS'])] = [(row['Date'], row['Completeness_pct'])]
  else:
    vax_22[int(row['FIPS'])].append((row['Date'], row['Completeness_pct']))

for index, row in census2021.iterrows():
  if row['FIPS'] == 'UNK':
    continue

  if int(row['FIPS']) not in vax_21:
    vax_21[int(row['FIPS'])] = [(row['Date'], row['Completeness_pct'])]
  else:
    vax_21[int(row['FIPS'])].append((row['Date'], row['Completeness_pct']))

for index, row in census2020.iterrows():
  if row['FIPS'] == 'UNK':
    continue

  if int(row['FIPS']) not in vax_20:
    vax_20[int(row['FIPS'])] = [(row['Date'], row['Completeness_pct'])]
  else:
    vax_20[int(row['FIPS'])].append((row['Date'], row['Completeness_pct']))

In [ ]:
population = dict(sorted(population.items()))
keys_population = list(population.keys())
values_population = list(population.values())
population_df = pd.DataFrame({'FIPS': keys_population, '# Population': values_population})
population_df.to_csv('population.csv', index=False)

## Dataset with all information per day

In [ ]:
# data - fips - # cases - # death - pct vaccination - population

In [3]:
data_22 = pd.read_csv ('us-counties-2022.csv')
data_21 = pd.read_csv ('us-counties-2021.csv')
data_20 = pd.read_csv ('us-counties-2020.csv')

census2022 = pd.read_csv ('vaccination_per_county_2022.csv')
census2021 = pd.read_csv ('vaccination_per_county_2021.csv')
census2020 = pd.read_csv ('vaccination_per_county_2020.csv')

In [5]:
cases_deaths_22 = {}
for index, row in data_22.iterrows():
  if np.isnan(row['fips']):
    continue
  if int(row['fips']) not in cases_deaths_22:
    cases_deaths_22[int(row['fips'])] = [(row['date'], row['cases'], row['deaths'])]
  else:
    cases_deaths_22[int(row['fips'])].append((row['date'], row['cases'], row['deaths']))


cases_deaths_21 = {}
for index, row in data_21.iterrows():
  if np.isnan(row['fips']):
    continue
  if int(row['fips']) not in cases_deaths_21:
    cases_deaths_21[int(row['fips'])] = [(row['date'], row['cases'], row['deaths'])]
  else:
    cases_deaths_21[int(row['fips'])].append((row['date'], row['cases'], row['deaths']))


cases_deaths_20 = {}
for index, row in data_20.iterrows():
  if np.isnan(row['fips']):
    continue
  if int(row['fips']) not in cases_deaths_20:
    cases_deaths_20[int(row['fips'])] = [(row['date'], row['cases'], row['deaths'])]
  else:
    cases_deaths_20[int(row['fips'])].append((row['date'], row['cases'], row['deaths']))

In [91]:
final_array_2020 = []
date_2020 = []
fips_2020 = []
cases_2020 = []
death_2020 = []
vax_2020 = []
pop_2020 = []

cd_keys = cases_deaths_20.keys()
for i in cd_keys:
  for j in range(len(cases_deaths_20[i])):
    if i in population:
      pop = population[i]
    else:
      pop = 0
    
    date = cases_deaths_20[i][j][0]
    cases = cases_deaths_20[i][j][1]
    death = cases_deaths_20[i][j][2]
    fips = i

    v = 0

    if i in vax_20:
      vax_fips = vax_20[i]
      for k in range(len(vax_fips)):
        if vax_fips[k][0] == date:
          v = vax_fips[k][1]
          break

    # final_array_2020.append([date, fips, cases, death, v, pop])
    date_2020.append(date)
    fips_2020.append(fips)
    cases_2020.append(cases)
    death_2020.append(death)
    vax_2020.append(v)
    pop_2020.append(pop)

final_df_2020 = pd.DataFrame({'Date': date_2020, 'FIPS': fips_2020, 
                              'Cases': cases_2020, 'Death': death_2020, 
                              'Vaccination': vax_2020, 'Population': pop_2020})

In [92]:
final_array_2021 = []
date_2021 = []
fips_2021 = []
cases_2021 = []
death_2021 = []
vax_2021 = []
pop_2021 = []

cd_keys = cases_deaths_21.keys()
for i in cd_keys:
  for j in range(len(cases_deaths_21[i])):
    if i in population:
      pop = population[i]
    else:
      pop = 0
    
    date = cases_deaths_21[i][j][0]
    cases = cases_deaths_21[i][j][1]
    death = cases_deaths_21[i][j][2]
    fips = i

    v = 0

    if i in vax_21:
      vax_fips = vax_21[i]
      for k in range(len(vax_fips)):
        if vax_fips[k][0] == date:
          v = vax_fips[k][1]
          break

    # final_array_2021.append([date, fips, cases, death, v, pop])
    date_2021.append(date)
    fips_2021.append(fips)
    cases_2021.append(cases)
    death_2021.append(death)
    vax_2021.append(v)
    pop_2021.append(pop)

final_df_2021 = pd.DataFrame({'Date': date_2021, 'FIPS': fips_2021, 
                              'Cases': cases_2021, 'Death': death_2021, 
                              'Vaccination': vax_2021, 'Population': pop_2021})

In [93]:
final_array_2022 = []
date_2022 = []
fips_2022 = []
cases_2022 = []
death_2022 = []
vax_2022 = []
pop_2022 = []

cd_keys = cases_deaths_22.keys()
for i in cd_keys:
  for j in range(len(cases_deaths_22[i])):
    if i in population:
      pop = population[i]
    else:
      pop = 0
    
    date = cases_deaths_22[i][j][0]
    cases = cases_deaths_22[i][j][1]
    death = cases_deaths_22[i][j][2]
    fips = i

    v = 0

    if i in vax_22:
      vax_fips = vax_22[i]
      for k in range(len(vax_fips)):
        if vax_fips[k][0] == date:
          v = vax_fips[k][1]
          break

    # final_array_2022.append([date, fips, cases, death, v, pop])
    date_2022.append(date)
    fips_2022.append(fips)
    cases_2022.append(cases)
    death_2022.append(death)
    vax_2022.append(v)
    pop_2022.append(pop)

final_df_2022 = pd.DataFrame({'Date': date_2022, 'FIPS': fips_2022, 
                              'Cases': cases_2022, 'Death': death_2022, 
                              'Vaccination': vax_2022, 'Population': pop_2022})

In [94]:
final_df_2020.to_csv('final_df_2020.csv', index=False)
final_df_2021.to_csv('final_df_2021.csv', index=False)
final_df_2022.to_csv('final_df_2022.csv', index=False)